## Project Overview



## Import Libraries

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
from __future__ import annotations

# Set the working directory to the project root
project_root = Path().resolve().parent
os.chdir(project_root)

# import local modules
import prospectivity_tools as pt

In [2]:
# todo--delete??

# Step 1: Define paths
parquet_path = Path(pt.settings.paths["parquet"])
html_path = Path(pt.settings.paths["interactive_html"])

# Step 2: Handle existing files
overwrite = True  # Set to True if you want to overwrite existing outputs
skip_existing = False  # Set to True if you want to skip processing if dataset_descriptions exist

if (parquet_path.exists() or html_path.exists()) and not overwrite:
    if skip_existing:
        gdf_cols_description("gdf_cols_descriptions already exist, skipping")
    else:
        raise Exception("gdf_cols_description files exist. Use overwrite=True or skip_existing=True.")


In [3]:
# Read in geopackage
gdf = gpd.read_file(pt.settings.paths["input_gpkg"], engine="pyogrio").to_crs(pt.settings.crs)

In [4]:
# gdf.explore()

In [5]:
# print(gdf.info())

In [6]:
# gdf_cols_description = pt.df_more_info(gdf)
# print(gdf_cols_description)

In [7]:
##PARAMETERS (todo)

# adjustable fall-off distance parameter. 
# Fuzzy Gamma? https://link.springer.com/article/10.1007/s11053-023-10255-8#:~:text=match%20at%20L755%20geophysical%20response,intermediate%20layers%20were%20then%20combined

# Notes: 
Good columns: 

    -upid--unique
    -area_m2--min 9322 max much bigger

questions

    -era
    -gid
    -strat_unit
    -period
    -strat_age
    -strat_name
    -gp_suite

In [8]:
# from llama_index.llms.openai import OpenAI
# from llama_index.core import Settings

# # Initialize the LLM
# llm = OpenAI(model="gpt-4o", temperature=0)

# # Set the LLM in global settings
# Settings.llm = llm

# prompt = f"""
# Describe the columns in the geopandas dataframe in plain english: 
# {str(gdf_cols_description + str(gdf.info()))[0:1000]} 
# Here is the complete info. Discuss your interpretation of what they mean. 
# Use regular line breaks in your response (encode wrapping) """

# # Input a string and get a string back
# response = llm.complete(prompt)
# print(response)

# ## Todo, finish and make clean

# Todo: clean up

Essential / high‑leverage columns
- geometry: shape data needed to calculate distances and overlays
- rock_type: specific lithology (e.g. granodiorite, ultramafic) used for proximity scoring
- rock_class: general rock group (e.g. sedimentary, metamorphic); useful backup
- unit_desc: full-text description with keywords like "serpentinite" or "granodiorite"
- strat_name: formal stratigraphic name; useful if rock_type is missing
- gp_suite: geological suite name; often overlaps with strat_name
- strat_unit: identifier for mapped unit; may carry lithological meaning
- area_m2: polygon area; helpful for QA on slivers or spatial joins

Potentially handy columns
- era: general geologic time range; not used directly in model
- period: finer temporal grouping; unused in this assignment
- strat_age: stratigraphic age (e.g. Upper Triassic); good for context
- age_min: minimum mapped age; not relevant to proximity rules
- age_max: maximum mapped age; same as above
- belt: tectonic belt (e.g. Coast); offers geologic context
- terrane: structural terrane (e.g. Quesnellia); useful for regional framing
- basin_age: age of basin fill; rare and unused
- rk_char: rarely used rock modifiers (e.g. “leuco”); low info content

Mostly metadata / least useful columns
- gid: numeric record ID; useful for joins only
- upid: unique hash-like ID; same as above
- project: mapping project name (e.g. “Okanagan”); not needed for modeling
- map_comp: author and date of compilation; metadata only
- src_url: link to source publication; citation only
- src_ref_s: text citation string; for documentation
- pub_org: publisher (always BCGS); not relevant to logic
- edit_date: constant value (“2018-04-05”); not informative
- basin: mostly None; no lithological utility
- mem_phase: sub-unit info; mostly empty
- fm_lithodm: formal formation info; mostly empty

In [9]:
## Todo clean up
## Feature engineering

# ------------------------------------------------------------------
# Lithology flags (ultramafic & granodiorite proximity drivers)
# ------------------------------------------------------------------
# Build a lowercase text blob for keyword searches
text_cols = ["rock_type", "unit_desc", "strat_name"]
gdf["_search"] = (
    gdf[text_cols]
    .fillna("")
    .agg(" | ".join, axis=1)
    .str.lower()
)

# Binary indicators for target lithologies
gdf["is_ultramafic"] = gdf["_search"].str.contains(
    r"(?:ultramafic|serpentinite|peridotite)"
).astype(int)

gdf["is_granodiorite"] = gdf["_search"].str.contains(
    r"\bgranodiorite\b"
).astype(int)

# ------------------------------------------------------------------
# Rock‑class back‑fill (simple heuristic)
# ------------------------------------------------------------------
class_map = {"granodiorite": "plutonic", "serpentinite": "metamorphic"}


# todo is this right??
def infer_class(row):
    if pd.isna(row["rock_class"]):
        for k, v in class_map.items():
            if k in row["_search"]:
                return v
    return row["rock_class"]

gdf["inferred_rock_class"] = gdf.apply(infer_class, axis=1)
gdf.drop(columns=["_search"], inplace=True)

# Keep track of generated columns
generated_cols = [
    "is_ultramafic",
    "is_granodiorite",
    "inferred_rock_class",
]

In [10]:
gen_columns_description = pt.df_more_info(gdf[generated_cols])
print(gen_columns_description)

Column: is_ultramafic
  Data type: int32


Column: is_granodiorite
  Data type: int32


Column: inferred_rock_class
  Value counts:
sedimentary rocks    556
intrusive rocks      455
volcanic rocks       435
metamorphic rocks    104
ultramafic rocks      46
Name: inferred_rock_class, dtype: int64





In [11]:
# # ------------------------------------------------------------------
# # 8. Improved visualization
# # ------------------------------------------------------------------
# fig, ax = plt.subplots(figsize=(12, 12))  # Larger figure for better detail
# grid.plot(
#     column="cobalt_score",
#     ax=ax,
#     cmap="plasma",           # Better contrast colormap
#     markersize=1,            # Smaller markers for higher resolution
#     scheme="quantiles",
#     legend=True
# )

# # Add geological context
# gdf.boundary.plot(ax=ax, color="white", linewidth=0.3, alpha=0.7)

# # Highlight target lithologies
# if len(gdf[gdf["is_ultramafic"] == 1]) > 0:
#     gdf[gdf["is_ultramafic"] == 1].plot(ax=ax, color="red", alpha=0.8, 
#                                         edgecolor="darkred", linewidth=0.5, label="Ultramafic")
# if len(gdf[gdf["is_granodiorite"] == 1]) > 0:
#     gdf[gdf["is_granodiorite"] == 1].plot(ax=ax, color="blue", alpha=0.8, 
#                                           edgecolor="darkblue", linewidth=0.5, label="Granodiorite")

# ax.set_title("Cobalt Prospectivity Heatmap\n(Exponential Decay Model - 500m Resolution)", 
#              fontsize=16, pad=20)
# ax.legend(loc='upper right')
# plt.axis("equal")
# plt.tight_layout()
# plt.show()

# # ------------------------------------------------------------------
# # 9. Score distribution analysis
# # ------------------------------------------------------------------
# print(f"\nScore Statistics:")
# print(grid["cobalt_score"].describe())
# print(f"Unique values: {grid['cobalt_score'].nunique()}")
# print(f"Grid points: {len(grid)}")

In [12]:
# import geopandas as gpd
# import numpy as np
# from shapely.geometry import Point
# import matplotlib.pyplot as plt

# # ------------------------------------------------------------------
# # 0.  PARAMETERS  (edit here)
# # ------------------------------------------------------------------
# ULTRAMAFIC_WEIGHT   = 2.0      # serpentinised / ultramafic carries most Co signal
# GRANODIORITE_WEIGHT = 1.0      # fertile intrusions carry secondary weight
# CELL_SIZE_M         = 2_000    # grid resolution (m) → 2 km pixels
# DECAY_DISTANCE_M    = 15_000   # distance at which influence decays to ~37% (1/e)
# CRS_EQUAL_AREA      = 3978     # NAD83 Canada Albers; swap if you work elsewhere
# # ------------------------------------------------------------------

# # 1.  Re‑project to an equal‑area CRS for distance work
# if gdf.crs is None or gdf.crs.is_geographic:
#     gdf = gdf.to_crs(CRS_EQUAL_AREA)

# # 2.  Build unified target geometries
# ultra_union  = gdf[gdf["is_ultramafic"] == 1].geometry.unary_union
# grano_union  = gdf[gdf["is_granodiorite"] == 1].geometry.unary_union

# # 3.  Create a fishnet grid that covers the study area
# xmin, ymin, xmax, ymax = gdf.total_bounds
# xs = np.arange(xmin, xmax + CELL_SIZE_M, CELL_SIZE_M)
# ys = np.arange(ymin, ymax + CELL_SIZE_M, CELL_SIZE_M)
# grid_points = [Point(x, y) for x in xs for y in ys]
# grid = gpd.GeoDataFrame(geometry=grid_points, crs=gdf.crs)

# # 4.  Compute distances to each target set (no capping needed with exponential decay)
# grid["d_ultra"] = grid.geometry.apply(lambda p: p.distance(ultra_union))
# grid["d_grano"] = grid.geometry.apply(lambda p: p.distance(grano_union))

# # 5.  Transform distances into scores using exponential decay
# #     This provides smooth, continuous scoring without hard cutoffs
# grid["s_ultra"] = ULTRAMAFIC_WEIGHT * np.exp(-grid["d_ultra"] / DECAY_DISTANCE_M)
# grid["s_grano"] = GRANODIORITE_WEIGHT * np.exp(-grid["d_grano"] / DECAY_DISTANCE_M)

# # 6.  Final prospectivity score
# grid["cobalt_score"] = grid["s_ultra"] + grid["s_grano"]

# # 7.  Classify into quantiles for mapping (now with much better distribution)
# try:
#     grid["score_class"] = pd.qcut(grid["cobalt_score"], q=5, labels=False)
#     print("Successfully used quantile-based classification")
# except ValueError as e:
#     print(f"Quantile classification failed: {e}")
#     print("Falling back to equal-width binning")
#     grid["score_class"] = pd.cut(grid["cobalt_score"], bins=5, labels=False)

# # Check the class distribution
# print("\nScore class distribution:")
# print(grid["score_class"].value_counts().sort_index())

# # ------------------------------------------------------------------
# # 8.  Quick visual sanity‑check
# # ------------------------------------------------------------------
# fig, ax = plt.subplots(figsize=(8, 8))
# grid.plot(column="cobalt_score",
#           ax=ax,
#           cmap="viridis",
#           markersize=8,
#           scheme="quantiles",
#           legend=True)
# gdf.boundary.plot(ax=ax, color="black", linewidth=0.2)
# ax.set_title("Cobalt prospectivity (exponential decay model)")
# plt.axis("equal")
# plt.show()

# # ------------------------------------------------------------------
# # 9.  Investigate the NEW score distribution
# # ------------------------------------------------------------------
# print("\nNEW Cobalt Score Statistics:")
# print(grid["cobalt_score"].describe())
# print(f"\nUnique values: {grid['cobalt_score'].nunique()}")
# print(f"Total points: {len(grid)}")

# # Check for the most common values
# print("\nMost common scores:")
# print(grid["cobalt_score"].value_counts().head(10))

# # Plot histogram
# plt.figure(figsize=(10, 6))
# plt.hist(grid["cobalt_score"], bins=50, alpha=0.7)
# plt.xlabel("Cobalt Score")
# plt.ylabel("Frequency")
# plt.title("Distribution of Cobalt Scores (Exponential Decay)")
# plt.show()

# # Optional: Show the decay curves
# plt.figure(figsize=(12, 4))

# # Subplot 1: Ultramafic influence
# plt.subplot(1, 2, 1)
# distances = np.linspace(0, 50000, 1000)
# influence = ULTRAMAFIC_WEIGHT * np.exp(-distances / DECAY_DISTANCE_M)
# plt.plot(distances/1000, influence, 'b-', linewidth=2)
# plt.xlabel('Distance (km)')
# plt.ylabel('Influence Score')
# plt.title('Ultramafic Influence vs Distance')
# plt.grid(True, alpha=0.3)

# # Subplot 2: Granodiorite influence
# plt.subplot(1, 2, 2)
# influence = GRANODIORITE_WEIGHT * np.exp(-distances / DECAY_DISTANCE_M)
# plt.plot(distances/1000, influence, 'r-', linewidth=2)
# plt.xlabel('Distance (km)')
# plt.ylabel('Influence Score')
# plt.title('Granodiorite Influence vs Distance')
# plt.grid(True, alpha=0.3)

# plt.tight_layout()
# plt.show()

In [13]:
##############################################################

In [14]:
# Load rock polygons
rock_a, rock_b = pt.load_bedrock(gdf)

In [15]:
#rock_a.explore(color="red", label="Ultramafic")
#rock_b.explore(color="blue", label="Granodiorite")
#gdf.explore(color="grey", label="Other Lithologies", alpha=0.1)

In [16]:
# Build the hexagon grid covering both rock types
grid = pt.build_grid(rock_a, rock_b)

In [17]:
# Reload the module to get the updated code
import importlib
importlib.reload(pt.viz)

<module 'prospectivity_tools.viz' from 'C:\\Users\\chris\\Documents\\GitHub\\prospectivity-heatmap\\src\\prospectivity_tools\\viz.py'>

In [18]:
# Compute distances to each rock type
grid = pt.add_distance_columns(grid, rock_a, rock_b, crs=pt.settings.crs)

In [19]:
# Calculate likelihood scores using the Gaussian kernel
scored = pt.compute_likelihood(grid)

In [22]:
scored

,h3_id,score


In [20]:
# Persist results
out_path = pt.write_parquet(scored)

# Create an interactive map
html_path = pt.build_map(scored)

In [21]:
html_path